# 1. Installing dependencies 


In [7]:
# !pip install -r requirements.txt
# !pip install --upgrade numpy
# !pip install --upgrade h5py


  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Audrey\\anaconda3\\Lib\\site-packages\\~5py\\defs.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



      Successfully uninstalled h5py-2.10.0


# 2. Model training 
The program applies Transfer Learning to this existing model and re-trains it to classify a new set of images.

This example shows how to take a Inception v3 architecture model trained on ImageNet images,
and train a new top layer that can recognize other classes of images.

You can replace the image_dir argument with any folder containing subfolders of
images. The label for each image is taken from the name of the subfolder it's in.



In [5]:
#TEMPORARY (removing datafiles from dataset to reduce amount of files)
# import os
# import random

# # Set the path to your dataset
# dataset_path = './dataset'

# # Set the number of images you want to delete from each class
# num_images_to_delete = 2500  # Change this to the number you want

# # Iterate over each class directory
# for class_dir in os.listdir(dataset_path):
#     class_dir_path = os.path.join(dataset_path, class_dir)

#     # Check if it's a directory
#     if os.path.isdir(class_dir_path):
#         # Get all images in the directory
#         images = [os.path.join(class_dir_path, img) for img in os.listdir(class_dir_path)]
#         # Shuffle the list to get random images to delete
#         random.shuffle(images)

#         # Delete the specified number of images
#         for img_to_delete in images[:num_images_to_delete]:
#             os.remove(img_to_delete)
#             print(f"Deleted {img_to_delete}")

# print("Deletion complete.")


Deleted ./dataset\A\A1993.jpg
Deleted ./dataset\A\A2340.jpg
Deleted ./dataset\A\A583.jpg
Deleted ./dataset\A\A29.jpg
Deleted ./dataset\A\A2153.jpg
Deleted ./dataset\A\A1695.jpg
Deleted ./dataset\A\A210.jpg
Deleted ./dataset\A\A900.jpg
Deleted ./dataset\A\A205.jpg
Deleted ./dataset\A\A2475.jpg
Deleted ./dataset\A\A2672.jpg
Deleted ./dataset\A\A190.jpg
Deleted ./dataset\A\A2224.jpg
Deleted ./dataset\A\A2785.jpg
Deleted ./dataset\A\A70.jpg
Deleted ./dataset\A\A249.jpg
Deleted ./dataset\A\A2646.jpg
Deleted ./dataset\A\A330.jpg
Deleted ./dataset\A\A2821.jpg
Deleted ./dataset\A\A2206.jpg
Deleted ./dataset\A\A42.jpg
Deleted ./dataset\A\A1447.jpg
Deleted ./dataset\A\A892.jpg
Deleted ./dataset\A\A2317.jpg
Deleted ./dataset\A\A1563.jpg
Deleted ./dataset\A\A301.jpg
Deleted ./dataset\A\A1855.jpg
Deleted ./dataset\A\A770.jpg
Deleted ./dataset\A\A2021.jpg
Deleted ./dataset\A\A1051.jpg
Deleted ./dataset\A\A2572.jpg
Deleted ./dataset\A\A153.jpg
Deleted ./dataset\A\A15.jpg
Deleted ./dataset\A\A1945.jpg

Deleted ./dataset\A\A129.jpg
Deleted ./dataset\A\A2815.jpg
Deleted ./dataset\A\A2378.jpg
Deleted ./dataset\A\A2968.jpg
Deleted ./dataset\A\A2423.jpg
Deleted ./dataset\A\A1351.jpg
Deleted ./dataset\A\A1897.jpg
Deleted ./dataset\A\A120.jpg
Deleted ./dataset\A\A1713.jpg
Deleted ./dataset\A\A1136.jpg
Deleted ./dataset\A\A130.jpg
Deleted ./dataset\A\A219.jpg
Deleted ./dataset\A\A360.jpg
Deleted ./dataset\A\A1118.jpg
Deleted ./dataset\A\A2705.jpg
Deleted ./dataset\A\A617.jpg
Deleted ./dataset\A\A159.jpg
Deleted ./dataset\A\A1515.jpg
Deleted ./dataset\A\A2518.jpg
Deleted ./dataset\A\A1498.jpg
Deleted ./dataset\A\A461.jpg
Deleted ./dataset\A\A2520.jpg
Deleted ./dataset\A\A1343.jpg
Deleted ./dataset\A\A2927.jpg
Deleted ./dataset\A\A2591.jpg
Deleted ./dataset\A\A1625.jpg
Deleted ./dataset\A\A2142.jpg
Deleted ./dataset\A\A1148.jpg
Deleted ./dataset\A\A731.jpg
Deleted ./dataset\A\A2227.jpg
Deleted ./dataset\A\A767.jpg
Deleted ./dataset\A\A1525.jpg
Deleted ./dataset\A\A1810.jpg
Deleted ./dataset\A\

Deleted ./dataset\A\A702.jpg
Deleted ./dataset\A\A2270.jpg
Deleted ./dataset\A\A1671.jpg
Deleted ./dataset\A\A2800.jpg
Deleted ./dataset\A\A1568.jpg
Deleted ./dataset\A\A2414.jpg
Deleted ./dataset\A\A814.jpg
Deleted ./dataset\A\A374.jpg
Deleted ./dataset\A\A2769.jpg
Deleted ./dataset\A\A1601.jpg
Deleted ./dataset\A\A1417.jpg
Deleted ./dataset\A\A2745.jpg
Deleted ./dataset\A\A813.jpg
Deleted ./dataset\A\A1146.jpg
Deleted ./dataset\A\A1818.jpg
Deleted ./dataset\A\A2148.jpg
Deleted ./dataset\A\A1681.jpg
Deleted ./dataset\A\A1383.jpg
Deleted ./dataset\A\A614.jpg
Deleted ./dataset\A\A2068.jpg
Deleted ./dataset\A\A102.jpg
Deleted ./dataset\A\A92.jpg
Deleted ./dataset\A\A760.jpg
Deleted ./dataset\A\A2049.jpg
Deleted ./dataset\A\A983.jpg
Deleted ./dataset\A\A1033.jpg
Deleted ./dataset\A\A1145.jpg
Deleted ./dataset\A\A2652.jpg
Deleted ./dataset\A\A2870.jpg
Deleted ./dataset\A\A2327.jpg
Deleted ./dataset\A\A1501.jpg
Deleted ./dataset\A\A1768.jpg
Deleted ./dataset\A\A1150.jpg
Deleted ./dataset\A\

Deleted ./dataset\A\A2381.jpg
Deleted ./dataset\A\A549.jpg
Deleted ./dataset\A\A2867.jpg
Deleted ./dataset\A\A1958.jpg
Deleted ./dataset\A\A493.jpg
Deleted ./dataset\A\A469.jpg
Deleted ./dataset\A\A1551.jpg
Deleted ./dataset\A\A2031.jpg
Deleted ./dataset\A\A1212.jpg
Deleted ./dataset\A\A428.jpg
Deleted ./dataset\A\A1516.jpg
Deleted ./dataset\A\A1011.jpg
Deleted ./dataset\A\A1359.jpg
Deleted ./dataset\A\A2614.jpg
Deleted ./dataset\A\A728.jpg
Deleted ./dataset\A\A764.jpg
Deleted ./dataset\A\A2942.jpg
Deleted ./dataset\A\A2779.jpg
Deleted ./dataset\A\A2104.jpg
Deleted ./dataset\A\A2859.jpg
Deleted ./dataset\A\A523.jpg
Deleted ./dataset\A\A2055.jpg
Deleted ./dataset\A\A2755.jpg
Deleted ./dataset\A\A339.jpg
Deleted ./dataset\A\A2322.jpg
Deleted ./dataset\A\A2701.jpg
Deleted ./dataset\A\A2042.jpg
Deleted ./dataset\A\A235.jpg
Deleted ./dataset\A\A575.jpg
Deleted ./dataset\A\A2375.jpg
Deleted ./dataset\A\A759.jpg
Deleted ./dataset\A\A2403.jpg
Deleted ./dataset\A\A2843.jpg
Deleted ./dataset\A\A

Deleted ./dataset\A\A555.jpg
Deleted ./dataset\A\A1044.jpg
Deleted ./dataset\A\A148.jpg
Deleted ./dataset\A\A37.jpg
Deleted ./dataset\A\A227.jpg
Deleted ./dataset\A\A972.jpg
Deleted ./dataset\A\A1413.jpg
Deleted ./dataset\A\A1786.jpg
Deleted ./dataset\A\A825.jpg
Deleted ./dataset\A\A2244.jpg
Deleted ./dataset\A\A2539.jpg
Deleted ./dataset\A\A1529.jpg
Deleted ./dataset\A\A985.jpg
Deleted ./dataset\A\A863.jpg
Deleted ./dataset\A\A2619.jpg
Deleted ./dataset\A\A1875.jpg
Deleted ./dataset\A\A2731.jpg
Deleted ./dataset\A\A2525.jpg
Deleted ./dataset\A\A2167.jpg
Deleted ./dataset\A\A2897.jpg
Deleted ./dataset\A\A1522.jpg
Deleted ./dataset\A\A1735.jpg
Deleted ./dataset\A\A1965.jpg
Deleted ./dataset\A\A2198.jpg
Deleted ./dataset\A\A736.jpg
Deleted ./dataset\A\A1752.jpg
Deleted ./dataset\A\A755.jpg
Deleted ./dataset\A\A917.jpg
Deleted ./dataset\A\A1308.jpg
Deleted ./dataset\A\A600.jpg
Deleted ./dataset\A\A2778.jpg
Deleted ./dataset\A\A1123.jpg
Deleted ./dataset\A\A2276.jpg
Deleted ./dataset\A\A28

...

Deleted ./dataset\Y\Y1613.jpg
Deleted ./dataset\Y\Y179.jpg
Deleted ./dataset\Y\Y2681.jpg
Deleted ./dataset\Y\Y1361.jpg
Deleted ./dataset\Y\Y1113.jpg
Deleted ./dataset\Y\Y210.jpg
Deleted ./dataset\Y\Y831.jpg
Deleted ./dataset\Y\Y2516.jpg
Deleted ./dataset\Y\Y2863.jpg
Deleted ./dataset\Y\Y2438.jpg
Deleted ./dataset\Y\Y1316.jpg
Deleted ./dataset\Y\Y2078.jpg
Deleted ./dataset\Y\Y2812.jpg
Deleted ./dataset\Y\Y529.jpg
Deleted ./dataset\Y\Y808.jpg
Deleted ./dataset\Y\Y350.jpg
Deleted ./dataset\Y\Y2956.jpg
Deleted ./dataset\Y\Y1523.jpg
Deleted ./dataset\Y\Y1132.jpg
Deleted ./dataset\Y\Y71.jpg
Deleted ./dataset\Y\Y64.jpg
Deleted ./dataset\Y\Y2881.jpg
Deleted ./dataset\Y\Y2981.jpg
Deleted ./dataset\Y\Y2422.jpg
Deleted ./dataset\Y\Y555.jpg
Deleted ./dataset\Y\Y491.jpg
Deleted ./dataset\Y\Y684.jpg
Deleted ./dataset\Y\Y2767.jpg
Deleted ./dataset\Y\Y2169.jpg
Deleted ./dataset\Y\Y178.jpg
Deleted ./dataset\Y\Y732.jpg
Deleted ./dataset\Y\Y1064.jpg
Deleted ./dataset\Y\Y921.jpg
Deleted ./dataset\Y\Y1522.

Deleted ./dataset\Y\Y2414.jpg
Deleted ./dataset\Y\Y1023.jpg
Deleted ./dataset\Y\Y1845.jpg
Deleted ./dataset\Y\Y517.jpg
Deleted ./dataset\Y\Y1321.jpg
Deleted ./dataset\Y\Y2824.jpg
Deleted ./dataset\Y\Y2624.jpg
Deleted ./dataset\Y\Y2616.jpg
Deleted ./dataset\Y\Y2034.jpg
Deleted ./dataset\Y\Y276.jpg
Deleted ./dataset\Y\Y691.jpg
Deleted ./dataset\Y\Y478.jpg
Deleted ./dataset\Y\Y349.jpg
Deleted ./dataset\Y\Y93.jpg
Deleted ./dataset\Y\Y1936.jpg
Deleted ./dataset\Y\Y587.jpg
Deleted ./dataset\Y\Y2698.jpg
Deleted ./dataset\Y\Y2511.jpg
Deleted ./dataset\Y\Y2865.jpg
Deleted ./dataset\Y\Y774.jpg
Deleted ./dataset\Y\Y1209.jpg
Deleted ./dataset\Y\Y2929.jpg
Deleted ./dataset\Y\Y2512.jpg
Deleted ./dataset\Y\Y1239.jpg
Deleted ./dataset\Y\Y2714.jpg
Deleted ./dataset\Y\Y180.jpg
Deleted ./dataset\Y\Y1712.jpg
Deleted ./dataset\Y\Y2193.jpg
Deleted ./dataset\Y\Y2552.jpg
Deleted ./dataset\Y\Y2055.jpg
Deleted ./dataset\Y\Y2647.jpg
Deleted ./dataset\Y\Y993.jpg
Deleted ./dataset\Y\Y2399.jpg
Deleted ./dataset\Y\Y

Deleted ./dataset\Z\Z129.jpg
Deleted ./dataset\Z\Z2944.jpg
Deleted ./dataset\Z\Z643.jpg
Deleted ./dataset\Z\Z1785.jpg
Deleted ./dataset\Z\Z907.jpg
Deleted ./dataset\Z\Z680.jpg
Deleted ./dataset\Z\Z1389.jpg
Deleted ./dataset\Z\Z523.jpg
Deleted ./dataset\Z\Z482.jpg
Deleted ./dataset\Z\Z1315.jpg
Deleted ./dataset\Z\Z94.jpg
Deleted ./dataset\Z\Z206.jpg
Deleted ./dataset\Z\Z491.jpg
Deleted ./dataset\Z\Z2263.jpg
Deleted ./dataset\Z\Z348.jpg
Deleted ./dataset\Z\Z2775.jpg
Deleted ./dataset\Z\Z278.jpg
Deleted ./dataset\Z\Z2082.jpg
Deleted ./dataset\Z\Z90.jpg
Deleted ./dataset\Z\Z2543.jpg
Deleted ./dataset\Z\Z2478.jpg
Deleted ./dataset\Z\Z1194.jpg
Deleted ./dataset\Z\Z1981.jpg
Deleted ./dataset\Z\Z1525.jpg
Deleted ./dataset\Z\Z601.jpg
Deleted ./dataset\Z\Z2284.jpg
Deleted ./dataset\Z\Z191.jpg
Deleted ./dataset\Z\Z1816.jpg
Deleted ./dataset\Z\Z1061.jpg
Deleted ./dataset\Z\Z1340.jpg
Deleted ./dataset\Z\Z1025.jpg
Deleted ./dataset\Z\Z2379.jpg
Deleted ./dataset\Z\Z2771.jpg
Deleted ./dataset\Z\Z2333.

Deleted ./dataset\Z\Z2047.jpg
Deleted ./dataset\Z\Z243.jpg
Deleted ./dataset\Z\Z1644.jpg
Deleted ./dataset\Z\Z2266.jpg
Deleted ./dataset\Z\Z1819.jpg
Deleted ./dataset\Z\Z557.jpg
Deleted ./dataset\Z\Z1671.jpg
Deleted ./dataset\Z\Z564.jpg
Deleted ./dataset\Z\Z645.jpg
Deleted ./dataset\Z\Z2909.jpg
Deleted ./dataset\Z\Z1362.jpg
Deleted ./dataset\Z\Z1860.jpg
Deleted ./dataset\Z\Z849.jpg
Deleted ./dataset\Z\Z555.jpg
Deleted ./dataset\Z\Z625.jpg
Deleted ./dataset\Z\Z29.jpg
Deleted ./dataset\Z\Z1029.jpg
Deleted ./dataset\Z\Z1379.jpg
Deleted ./dataset\Z\Z2866.jpg
Deleted ./dataset\Z\Z1244.jpg
Deleted ./dataset\Z\Z1972.jpg
Deleted ./dataset\Z\Z507.jpg
Deleted ./dataset\Z\Z428.jpg
Deleted ./dataset\Z\Z642.jpg
Deleted ./dataset\Z\Z481.jpg
Deleted ./dataset\Z\Z1053.jpg
Deleted ./dataset\Z\Z1043.jpg
Deleted ./dataset\Z\Z1124.jpg
Deleted ./dataset\Z\Z714.jpg
Deleted ./dataset\Z\Z2861.jpg
Deleted ./dataset\Z\Z2344.jpg
Deleted ./dataset\Z\Z1152.jpg
Deleted ./dataset\Z\Z1483.jpg
Deleted ./dataset\Z\Z441

Deleted ./dataset\Z\Z2734.jpg
Deleted ./dataset\Z\Z2576.jpg
Deleted ./dataset\Z\Z1079.jpg
Deleted ./dataset\Z\Z1576.jpg
Deleted ./dataset\Z\Z767.jpg
Deleted ./dataset\Z\Z1278.jpg
Deleted ./dataset\Z\Z1393.jpg
Deleted ./dataset\Z\Z2034.jpg
Deleted ./dataset\Z\Z2955.jpg
Deleted ./dataset\Z\Z2094.jpg
Deleted ./dataset\Z\Z2688.jpg
Deleted ./dataset\Z\Z2897.jpg
Deleted ./dataset\Z\Z330.jpg
Deleted ./dataset\Z\Z2343.jpg
Deleted ./dataset\Z\Z179.jpg
Deleted ./dataset\Z\Z898.jpg
Deleted ./dataset\Z\Z1801.jpg
Deleted ./dataset\Z\Z594.jpg
Deleted ./dataset\Z\Z2635.jpg
Deleted ./dataset\Z\Z597.jpg
Deleted ./dataset\Z\Z931.jpg
Deleted ./dataset\Z\Z2722.jpg
Deleted ./dataset\Z\Z915.jpg
Deleted ./dataset\Z\Z2999.jpg
Deleted ./dataset\Z\Z1654.jpg
Deleted ./dataset\Z\Z2309.jpg
Deleted ./dataset\Z\Z877.jpg
Deleted ./dataset\Z\Z725.jpg
Deleted ./dataset\Z\Z1249.jpg
Deleted ./dataset\Z\Z384.jpg
Deleted ./dataset\Z\Z735.jpg
Deleted ./dataset\Z\Z1527.jpg
Deleted ./dataset\Z\Z521.jpg
Deleted ./dataset\Z\Z16

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Assuming there are 29 classes based on the folders A-Z, del, nothing, space
your_number_of_classes = 29
path_to_your_train_data = './training_dataset'  # Update this path to where your dataset is located

# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Freeze the base model
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # New FC layer, random init
predictions = Dense(your_number_of_classes, activation='softmax')(x)  # New softmax layer
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    path_to_your_train_data,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# Train the model
model.fit(train_generator, steps_per_epoch=train_generator.n // 32, epochs=10)

# Optionally, fine-tune the model by unfreezing layers and recompiling the model


C:\Users\Audrey\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


9406464/9406464 [==============================] - 2s 0us/step


C:\Users\Audrey\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Found 14210 images belonging to 29 classes.
Epoch 1/10
444/444 [==============================] - 277s 618ms/step - loss: 1.5516 - accuracy: 0.6081
Epoch 2/10
444/444 [==============================] - 270s 607ms/step - loss: 0.6587 - accuracy: 0.8345
Epoch 3/10
444/444 [==============================] - 271s 610ms/step - loss: 0.5013 - accuracy: 0.8637
Epoch 4/10
444/444 [==============================] - 267s 601ms/step - loss: 0.4283 - accuracy: 0.8792
Epoch 5/10
444/444 [==============================] - 276s 622ms/step - loss: 0.3649 - accuracy: 0.8954
Epoch 6/10
444/444 [==============================] - 279s 628ms/step - loss: 0.3441 - accuracy: 0.9024
Epoch 7/10
444/444 [==============================] - 263s 591ms/step - loss: 0.3020 - accuracy: 0.9142
Epoch 8/10
444/444 [==============================] - 268s 604ms/step - loss: 0.2850 - accuracy: 0.9145
Epoch 9/10
444/444 [==============================] - 284s 639ms/step - loss: 0.2673 - accuracy: 0.9209
Epoch 10/10
444/444 

# 3. Classification

In [5]:
import os

def classify_folder(folder_path, model, generator):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            img_array = prepare_image(image_path)
            predictions = model.predict(img_array)
            class_index = np.argmax(predictions[0])
            class_label = get_class_label(class_index, generator)
            probability = np.max(predictions[0])
            print(f"Image: {filename} - Prediction: {class_label} (Score = {probability:.5f})")

# Path to your folder containing images
folder_path = './testimages'  # Update this to your folder path

# Classify all images in the folder
classify_folder(folder_path, model, train_generator)


1/1 [==============================] - 0s 32ms/step
Image: testimage1.jpg - Prediction: D (Score = 0.99878)
1/1 [==============================] - 0s 31ms/step
Image: testimage2.jpg - Prediction: del (Score = 0.99819)


# 4. Webcam (not now)